In [1]:
import basis_set_exchange as bse
import matplotlib.pyplot as plt
from pyscf import gto,scf
import numpy as np
from scipy.interpolate import interp1d
from pyscf.data.elements import _symbol
import copy

In [2]:
from FcMole import FcM, FcM_like
from FDcoeffs import get_coeffs

In [3]:
fd1=np.asarray([1/12,-2/3 ,0,2/3,-1/12])
fd2=np.asarray([-1/12,4/3,-5/2,4/3,-1/12])
fd3=np.asarray([-1/2,1,0,-1,1/2])
fd4=np.asarray([1,-4,6,-4,1])
fds=[fd1,fd2,fd3,fd4]
fds5=get_coeffs(5)
fds7=get_coeffs(7)

This is a quadratic interpolation of the basis set coefficients

In [12]:
def FCAO(z,bs_name="pcX-1",ref=None):
    if z <1.e-10: raise Exception ("non implemented Zero charge atom")
    if z%1 <1.e-10:
        return gto.basis.load(bse.get_basis(bs_name,fmt="nwchem",elements=[int(z)]),_symbol(int(z)))
    elif ref:
        rb=FCAO(ref,bs_name)
        rbm=FCAO(ref-1,bs_name)
        rbp=FCAO(ref+1,bs_name)
        fb=copy.deepcopy(rb)
        for i in range(len(fb)):  # ref+a(z-zr) +b(z-zr)^2
            a=(rbp[i][1][0]-rbm[i][1][0])/2
            b=(rbp[i][1][0]+rbm[i][1][0]-2*rb[i][1][0])
            fb[i][1][0]+=a*(z-ref)+b*(z-ref)**2
        return fb
    else: raise Exception("can't get the fractional charge basis set, frac val and no ref")

In [13]:
FCAO(6),FCAO(6.3,ref=6),FCAO(6.3,ref=7),FCAO(7)

([[0, [1477.26, 1.0]],
  [0, [222.361, 1.0]],
  [0, [50.5229, 1.0]],
  [0, [13.9594, 1.0]],
  [0, [4.21475, 1.0]],
  [0, [0.653345, 1.0]],
  [0, [0.191501, 1.0]],
  [1, [10.9571, 1.0]],
  [1, [2.32904, 1.0]],
  [1, [0.633257, 1.0]],
  [1, [0.16315, 1.0]],
  [2, [0.848528, 1.0]]],
 [[0, [1630.782, 1.0]],
  [0, [245.45781999999997, 1.0]],
  [0, [55.773661999999995, 1.0]],
  [0, [15.424070799999999, 1.0]],
  [0, [4.6667834, 1.0]],
  [0, [0.7296017599999999, 1.0]],
  [0, [0.21275233999999998, 1.0]],
  [1, [12.13559, 1.0]],
  [1, [2.5907785999999997, 1.0]],
  [1, [0.70331618, 1.0]],
  [1, [0.17980665999999998, 1.0]],
  [2, [0.8785770199999999, 1.0]]],
 [[0, [1646.4856, 1.0]],
  [0, [247.81945999999994, 1.0]],
  [0, [56.31019, 1.0]],
  [0, [15.574876, 1.0]],
  [0, [4.7146668, 1.0]],
  [0, [0.7381517, 1.0]],
  [0, [0.21519793999999998, 1.0]],
  [1, [12.205006, 1.0]],
  [1, [2.6080378, 1.0]],
  [1, [0.70727308, 1.0]],
  [1, [0.18007438, 1.0]],
  [2, [0.87857058, 1.0]]],
 [[0, [2004.67, 1.0]],


In [14]:
nn=gto.M(atom="N 0 0 0; N 0 0 2.05",unit="Bohr",basis=bse.get_basis("pcX-1",fmt="nwchem",elements=[6,7,8]))
mf=scf.RHF(nn)
e_nn=mf.scf()

converged SCF energy = -108.916591616523


In [15]:
nn=gto.M(atom="N 0 0 0; N 0 0 2.05",unit="Bohr",basis={7:FCAO(7,"pcX-2")})
mf=scf.RHF(nn)
e_nn=mf.scf()

converged SCF energy = -108.987830318743


In [16]:
hc_nn=FcM_like(nn,fcs=[.5,-.5])
print(hc_nn.nelec)
mfh=scf.RHF(hc_nn)
mfh.scf(dm0=mfh.init_guess_by_1e())

(7, 7)
converged SCF energy = -109.936895786835


-109.93689578683507

In [23]:
hc_nn_2=FcM(atom="N@1 0 0 0; N@2 0 0 2.05",fcs=[.5,-.5],unit="Bohr",\
            basis={"N@1":FCAO(7.5,"pcX-2",ref=7),"N@2":FCAO(6.5,"pcX-2",ref=7)})
print(hc_nn_2.atom_charges())
mfh=scf.RHF(hc_nn_2)

mfh.scf(dm0=mfh.init_guess_by_1e())

[7.5 6.5]
converged SCF energy = -109.938508227221


-109.93850822722081

In [19]:
cn=gto.M(atom="C 0 0 0; N 0 0 2.05",unit="Bohr",basis=bse.get_basis("pcX-1",fmt="nwchem",elements=[6,7,8]),charge=-1)
on=gto.M(atom="O 0 0 0; N 0 0 2.05",unit="Bohr",basis=bse.get_basis("pcX-1",fmt="nwchem",elements=[6,7,8]),charge=1)

In [51]:
%load_ext autoreload
%autoreload 2
from AP_class import APDFT_perturbator as AP

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
ap=AP(mfh,sites=[0,1])
ap.build_cubic_hessian()

array([[[-0.25690744,  0.10506625],
        [ 0.10506625,  0.12935436]],

       [[ 0.10506625,  0.12935436],
        [ 0.12935436, -0.24371954]]])

In [55]:
ap.build_gradient()
ap.build_hessian()

array([[-3.49057654,  0.48807414],
       [ 0.48807414, -3.11515008]])

In [56]:
ap.APDFT1(np.array([.5,-.5]))

-111.83757808221642

In [57]:
ap.APDFT1(np.array([-.5,.5]))

-108.03943837222519

In [59]:
ap.APDFT3(np.array([-.5,.5])),mf.e_tot

(-108.98841599354571, -108.98783031874271)